# Отчет по лабораторной работе #4
### Выполнила: Анна Казакова

Исследование проводилось над выборками, содержащимися в библиотеке Keras: imdb и boston_housing для классификации и регрессии

### Описание выборок:
#### IMDB: 
* Множество состоит из 50 000 самых разных отзывов к кинолентам. Набор разбит на 25 000 обучающих и 25 000 контрольных отзывов, каждый набор на 50% состоит из отрицательных и на 50 % из положительных отзывов.
* Отзывы уже преобразованы в последовательности целых чисел, каждое из которых определяет позицию слова в словаре. 
* При обучении модели использовал только 5000 самых часто встречающихся отзывов из-за ограничений железа.

#### Boston Housing: 
* Содержит относительно немного образцов данных: всего 506, разбитых на 404 обучающих и 102 контрольных образца
* Каждый признак во входных данных имеет свой масштаб
* Содержит 13 числовых признаков
* Цены в основной массе находятся в диапазоне от 10 000 до 50 000 долларов США

Для построения классификатора нам понадобятся следующие библиотеки и их объекты

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from keras import backend as K
from keras import layers, models
from keras.datasets import imdb

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

Для вычисления значений recall и precision были описаны данные функции

In [3]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

Эта функция кодирует последовательности целых чисел в бинарную матрицу для последующей передачи в нейронную сеть

In [4]:
def vectorize_sequences(sequences, dimension=5000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

Установим количество используемых слов, разделим выборку на тренировочные и тестовые части и преобразуем данные перед подачей в нейронную сеть

In [5]:
num_words = 5000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=num_words)
X_train = vectorize_sequences(train_data)
y_train = np.asarray(train_labels).astype('float32')
X_test = vectorize_sequences(test_data)
y_test = np.asarray(test_labels).astype('float32')

С нашей задачей прекрасно справится сеть, организованные как простой стек полносвязных слоев с операцией активации relu. Промежуточные слои будут использовать операцию relu в качестве функции активации, а последний слой будет использовать сигмоидную функцию активации и выводить вероятность. Функция relu используется для преобразования отрицательных значений в ноль. Так как перед нами стоит задача бинарной классификации и результатом работы сети является вероятность, предпочтительнее использовать функцию потерь binary_crossentropy

In [6]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(num_words,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc', recall, precision])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Создадим проверочный набор, выбрав 5000 образцов из
оригинального набора обучающих данных. Теперь проведем обучение модели в течение 20 эпох пакетами по 512 образцов. В то же время будем следить за потерями и точностью на 5000 отложенных образцов

In [7]:
x_val = X_train[:num_words]
partial_x_train = X_train[num_words:]
y_val = y_train[:num_words]
partial_y_train = y_train[num_words:]
history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))
history_dict = history.history
epochs = range(1, len(history_dict['acc'])+1)


Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 2s 91us/step - loss: 0.4744 - acc: 0.7952 - recall: 0.7895 - precision: 0.8160 - val_loss: 0.3419 - val_acc: 0.8750 - val_recall: 0.8771 - val_precision: 0.8768
Epoch 2/20
20000/20000 [==============================] - 1s 71us/step - loss: 0.2892 - acc: 0.8954 - recall: 0.9026 - precision: 0.8918 - val_loss: 0.3010 - val_acc: 0.8788 - val_recall: 0.8395 - val_precision: 0.9146
Epoch 3/20
20000/20000 [==============================] - 1s 72us/step - loss: 0.2366 - acc: 0.9133 - recall: 0.9211 - precision: 0.9075 - val_loss: 0.3139 - val_acc: 0.8720 - val_recall: 0.8110 - val_precision: 0.9288
Epoch 4/20
20000/20000 [==============================] - 1s 70us/step - loss: 0.2076 - acc: 0.9232 - recall: 0.9271 - precision: 0.9213 - val_loss: 0.2969 - val_acc: 0.8852 - val_recall: 0.9386 - val_precision: 0.8504
Epoch 5/20
20000/20000 [==============================] - 1s 73us/step - l

Заметим, что на этапе обучения потери снижаются с каждой эпохой, а точность растет. Но это не относится к потерям и точности на этапе проверки: похоже, что они достигли пика в четвертую эпоху. Соответственно можно утверждать, что в данном случае наблюдается переобучение. В данном случае для предотвращения переобучения можно прекратить обучение после третьей эпохи

In [1]:
plt.plot(epochs, history_dict['recall'], 'bo', label='recall')
plt.plot(epochs, history_dict['val_recall'], 'b', label='val_recall')
plt.xlabel('epoch')
plt.ylabel('recall')
plt.legend()
plt.show()
plt.clf()

plt.plot(epochs, history_dict['precision'], 'bo', label='precision')
plt.plot(epochs, history_dict['val_precision'], 'b', label='val_precision')
plt.xlabel('epoch')
plt.ylabel('precision')
plt.legend()
plt.show()
plt.clf()

plt.plot(epochs, history_dict['acc'], 'bo', label='accuracy')
plt.plot(epochs, history_dict['val_acc'], 'b', label='val_accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.clf()

NameError: name 'plt' is not defined

Теперь обучим новую сеть с нуля в течение четырех эпох и затем оценим получившийся результат на контрольных данных. В итоге мы достигли точности в ~88%, что является хорошим результатов для такой простой модели

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(num_words,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=4, batch_size=512)
model.save('imdb.h5')
print(model.evaluate(X_test, y_test))

Теперь перейдем к задаче регрессии. Библиотеки и их объекты, которые понадобятся нам для построения этой модели

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import boston_housing
from keras import models
from keras import layers

Нормализуем данные перед передачей в нейронную сеть: для каждого признака во входных данных из каждого значения вычитается среднее по
этому признаку, и разность делится на стандартное отклонение, в результате признак центрируется по нулевому значению и имеет стандартное отклонение

In [ ]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

Далее опишем функцию построения нашей сети: она заканчивается одномерным слоем, не имеющим функции активации. Это типичная конфигурация для скалярной регрессии. Функция mse широко используется
в задачах регрессии. Также мы включили мониторинг на абсолютное значение разности между предсказанными и целевыми значениями (mae)

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

Так как размерность нашего набора слишком мала для разбивки на обучающий и проверочный наборы лучшей практикой в таких ситуациях является применение перекрестной проверки по K блокам. В данном случае
средняя ошибка составила 3000 долларов, что довольно много

In [ ]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)

    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)
print(np.mean(all_scores))

Теперь попробуем увеличить время обучения сети до 500 эпох. Чтобы получить информацию о качестве обучения модели в каждую эпоху, изменим цикл обучения и добавим сохранение оценки проверки перед началом эпохи.

In [ ]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples:(i + 1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets, validation_data=(val_data, val_targets), epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

Построим график mae, исключая первые 10 замеров

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points


smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('epochs')
plt.ylabel('MAE')
plt.show()

Обучим окончательную версию модели и получим результат нашей работы. К сожалению средняя ошибка все еще составляет около 2550 долларов

In [ ]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mse_score, test_mae_score)

Выводы:
В данных примерах да и в целом перед построением и обучением моделей на каких-либо данных существует необходимость их преобразования в тензоры - векторы числовых данных признаков (в задаче регрессии их пришлось нормализовать). Также мы убедились, что по мере улучшения на обучающих данных нейронные сети рано или поздно
начинают переобучаться, демонстрируя ухудшение результатов на данных, которые они прежде не видели. За этим нужно следить и использовать известные методы борьбы с этим. Во избежание переобучения при небольших объемах данных следует использовать небольшие сети. При небольшом объемы данных как в нашей задаче регрессии для надежной оценки качества модели следует использовать метод перекрестной проверки по K блокам